In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import AdamW
from torch.cuda.amp import GradScaler, autocast

from pathlib import Path
from PIL import Image
import numpy as np
import json
from tqdm import tqdm
import os
import copy
from types import SimpleNamespace
import urllib.request
import zipfile

# --- 프로젝트 경로 추가 및 모듈 임포트 ---
import sys
sys.path.append('.')

# pycocotools와 torchmetrics가 필요합니다.
# pip install pycocotools torchmetrics
from pycocotools.coco import COCO
from torchmetrics.detection import MeanAveragePrecision

from models.blade_model_v2 import BladeModelV2
from utils.criterion import SetCriterion
from utils.hungarian_matcher import HungarianMatcher

# --- COCO 2017 Validation 데이터셋 자동 다운로드 ---
print("--- Setting up COCO 2017 Validation Dataset ---")
data_dir = Path('./data/coco')
data_dir.mkdir(parents=True, exist_ok=True)

# Annotations
if not (data_dir / 'annotations' / 'instances_val2017.json').exists():
    print("Downloading annotations...")
    url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
    urllib.request.urlretrieve(url, data_dir / 'annotations.zip')
    with zipfile.ZipFile(data_dir / 'annotations.zip', 'r') as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(data_dir / 'annotations.zip')
    print("Annotations downloaded and extracted.")

# Images
if not (data_dir / 'val2017').exists() or len(os.listdir(data_dir / 'val2017')) == 0:
    print("Downloading images (approx. 1GB)... This may take a while.")
    url = "http://images.cocodataset.org/zips/val2017.zip"
    urllib.request.urlretrieve(url, data_dir / 'val2017.zip')
    with zipfile.ZipFile(data_dir / 'val2017.zip', 'r') as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(data_dir / 'val2017.zip')
    print("Images downloaded and extracted.")

print("✅ COCO Dataset is ready.")

In [ ]:
# ===== 셀 2: COCO 테스트용 설정 (수정) =====

class CocoConfig:
    DATA_ROOT = Path('./data/coco')
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    BATCH_SIZE = 4
    EPOCHS = 2
    LR = 1e-4
    WEIGHT_DECAY = 1e-4
    NUM_WORKERS = 0

    # --- [핵심 수정] ---
    # 딕셔너리를 SimpleNamespace로 변경하여 점(.)으로 접근 가능하게 수정
    MODEL = SimpleNamespace(
        BACKBONE=SimpleNamespace(NAME='ConvNeXt-Tiny'),
        FPN=SimpleNamespace(OUT_CHANNELS=256),
        HEAD_B=SimpleNamespace(
            FEAT_CHANNELS=256,
            OUT_CHANNELS=256,
            NUM_CLASSES=80,
            QUERIES_PER_CLASS=5,
            DEC_LAYERS=6
        )
    )
    LOSS = SimpleNamespace(
        CLASS_WEIGHTS=[1.0] * 80,
        EOS_COEF=0.1
    )

config = CocoConfig()
print("--- COCO Test Configuration Initialized ---")

In [ ]:
class CocoDataset(Dataset):
    def __init__(self, root, split='val', transform=None):
        self.root = Path(root)
        self.split = split
        self.images_dir = self.root / f"{self.split}2017"
        self.coco = COCO(self.root / 'annotations' / f"instances_{self.split}2017.json")
        self.ids = list(sorted(self.coco.imgs.keys()))
        
        # COCO의 91개 카테고리를 0-79 인덱스로 매핑
        self.cat_ids = sorted(self.coco.getCatIds())
        self.cat2cat = {cat_id: i for i, cat_id in enumerate(self.cat_ids)}

        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((640, 640)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = self.images_dir / img_info['file_name']
        image = Image.open(img_path).convert('RGB')
        
        target = {}
        masks = []
        labels = []
        
        for ann in anns:
            if ann['iscrowd'] == 0:
                masks.append(self.coco.annToMask(ann))
                labels.append(self.cat2cat[ann['category_id']])

        image = self.transform(image)
        
        target['labels'] = torch.tensor(labels, dtype=torch.int64)
        if masks:
            target['masks'] = torch.from_numpy(np.stack(masks)).float()
        else:
            target['masks'] = torch.zeros((0, img_info['height'], img_info['width']), dtype=torch.float32)
            
        # 우리 모델과의 호환성을 위해 blade_mask 필드 추가 (여기서는 사용되지 않음)
        target['blade_mask'] = torch.zeros(image.shape[1:], dtype=torch.long)
            
        return image, target

def collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    images = torch.stack(images, dim=0)
    return images, targets

# COCO val 데이터셋을 train/val로 나눠서 테스트
full_dataset = CocoDataset(root=config.DATA_ROOT, split='val')
train_size = int(0.1 * len(full_dataset)) # 10%만 빠르게 테스트
val_size = int(0.02 * len(full_dataset)) # 2%만 검증
train_subset, val_subset, _ = torch.utils.data.random_split(
    full_dataset, [train_size, val_size, len(full_dataset) - train_size - val_size]
)

train_loader = DataLoader(
    train_subset, batch_size=config.BATCH_SIZE, shuffle=True,
    num_workers=config.NUM_WORKERS, collate_fn=collate_fn
)
val_loader = DataLoader(
    val_subset, batch_size=config.BATCH_SIZE, shuffle=False,
    num_workers=config.NUM_WORKERS, collate_fn=collate_fn
)
print("✅ COCO DataLoaders created!")

In [ ]:
# 모델 초기화 (Head-A는 사용하지 않으므로, 손실 계산 시 무시)
model = BladeModelV2(config).to(config.DEVICE)
matcher = HungarianMatcher(cost_class=2.0, cost_mask=5.0, cost_dice=5.0)
weight_dict = {'loss_ce': 2.0, 'loss_mask': 5.0, 'loss_dice': 5.0}
criterion = SetCriterion(
    num_classes=config.MODEL.HEAD_B.NUM_CLASSES, matcher=matcher, weight_dict=weight_dict,
    eos_coef=config.LOSS.EOS_COEF, losses=['labels', 'masks'],
    class_weights=config.LOSS.CLASS_WEIGHTS
).to(config.DEVICE)
optimizer = AdamW(model.head_b.parameters(), lr=config.LR, weight_decay=config.WEIGHT_DECAY) # Head-B만 학습
scaler = GradScaler()

def train_epoch_coco(model, criterion, dataloader, optimizer, device, epoch):
    model.train()
    model.head_b.train()
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"COCO Test Epoch {epoch+1}/{config.EPOCHS} [Train]")
    for images, targets in pbar:
        images = images.to(device)
        targets_gpu = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with autocast():
            outputs = model(images)
            loss_dict = criterion(outputs, targets_gpu)
            weighted_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        optimizer.zero_grad()
        scaler.scale(weighted_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += weighted_loss.item()
        pbar.set_postfix({'loss': f'{weighted_loss.item():.4f}'})
    return total_loss / len(dataloader)

def validate_coco(model, criterion, dataloader, device):
    model.eval()
    map_metric = MeanAveragePrecision(iou_type="segm")
    val_losses = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="[Valid]")
        for images, targets in pbar:
            images = images.to(device)
            targets_gpu = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            with autocast():
                outputs = model(images)
                loss_dict = criterion(outputs, targets_gpu)
                weighted_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
            val_losses.append(weighted_loss.item())

            # --- [수정] 이 부분이 생략되었던 곳입니다 ---
            # 모델의 GPU 출력값을 CPU로 가져옴
            pred_logits_cpu = outputs['pred_logits'].cpu()
            pred_masks_cpu = outputs['pred_masks'].cpu()
            
            # 1. Prediction을 torchmetrics 형식으로 변환
            preds_for_map = []
            for i in range(len(targets)):
                scores, labels = F.softmax(pred_logits_cpu[i], dim=-1).max(-1)
                masks_bool = (torch.sigmoid(pred_masks_cpu[i]) > 0.5)
                preds_for_map.append(dict(
                    masks=masks_bool,
                    scores=scores,
                    labels=labels,
                ))

            # 2. Target을 torchmetrics 형식으로 변환
            targets_for_map = []
            for t in targets:
                # CPU로 옮길 필요 없음 (targets는 이미 CPU에 있음)
                targets_for_map.append(dict(
                    masks=(t['masks'] > 0.5), # boolean mask로 변환
                    labels=t['labels'],
                ))
            
            map_metric.update(preds_for_map, targets_for_map)
            
    map_results = map_metric.compute()
    return {'loss': np.mean(val_losses), 'mAP': map_results['map'].item()}

print("✅ COCO Test functions are ready.")

In [ ]:
print("\n--- 🚀 Starting COCO Sanity Check Training 🚀 ---")
for epoch in range(config.EPOCHS):
    train_loss = train_epoch_coco(model, criterion, train_loader, optimizer, config.DEVICE, epoch)
    val_metrics = validate_coco(model, criterion, val_loader, config.DEVICE)
    print(f"\nEpoch {epoch+1}/{config.EPOCHS} -> Train Loss: {train_loss:.4f}, Val Loss: {val_metrics['loss']:.4f}, mAP: {val_metrics['mAP']:.4f}")

print("\n--- 🎉 COCO Test Complete ---")

Mask R CNN Test

In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.utils.data import DataLoader
from pathlib import Path
from PIL import Image
import numpy as np
import json
from tqdm import tqdm
from types import SimpleNamespace

# --- pycocotools와 torchmetrics가 필요합니다 ---
from pycocotools.coco import COCO
from torchmetrics.detection import MeanAveragePrecision

class CocoDataset(Dataset):
    def __init__(self, root, split='val'):
        self.root = Path(root)
        self.split = split
        self.images_dir = self.root / f"{self.split}2017"
        self.coco = COCO(self.root / 'annotations' / f"instances_{self.split}2017.json")
        self.ids = list(sorted(self.coco.imgs.keys()))
        
        self.cat_ids = sorted(self.coco.getCatIds())
        self.coco_labels_map = {cat_id: i + 1 for i, cat_id in enumerate(self.cat_ids)}

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        coco_anns = self.coco.loadAnns(ann_ids)
        
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = self.images_dir / img_info['file_name']
        
        # --- [수정] 이미지를 PIL Image 객체로 그대로 반환 (transform 제거) ---
        image = Image.open(img_path).convert('RGB')
        
        target = {}
        masks, labels, boxes = [], [], []

        for ann in coco_anns:
            if ann['iscrowd'] == 0 and ann['area'] > 0:
                masks.append(self.coco.annToMask(ann))
                labels.append(self.coco_labels_map[ann['category_id']])
                x, y, w, h = ann['bbox']
                boxes.append([x, y, x + w, y + h])
        
        if boxes:
            target['boxes'] = torch.as_tensor(boxes, dtype=torch.float32)
            target['labels'] = torch.as_tensor(labels, dtype=torch.int64)
            target['masks'] = torch.as_tensor(np.stack(masks), dtype=torch.uint8)
        else: 
            target['boxes'] = torch.zeros((0, 4), dtype=torch.float32)
            target['labels'] = torch.zeros((0,), dtype=torch.int64)
            target['masks'] = torch.zeros((0, img_info['height'], img_info['width']), dtype=torch.uint8)
            
        return image, target

# --- [수정] collate_fn에서 ToTensor 변환을 담당 ---
def collate_fn(batch):
    images = []
    targets = []
    to_tensor = transforms.ToTensor()
    for item in batch:
        # 각 PIL 이미지를 텐서로 변환하여 리스트에 추가
        images.append(to_tensor(item[0]))
        targets.append(item[1])
    return images, targets

# COCO val 데이터셋을 train/val로 나눠서 테스트
full_dataset = CocoDataset(root=config.DATA_ROOT, split='val')
train_size = int(0.1 * len(full_dataset)) # 10%만 빠르게 테스트
val_size = int(0.02 * len(full_dataset)) # 2%만 검증
train_subset, val_subset, _ = torch.utils.data.random_split(
    full_dataset, [train_size, val_size, len(full_dataset) - train_size - val_size]
)

train_loader = DataLoader(
    train_subset, batch_size=config.BATCH_SIZE, shuffle=True,
    num_workers=config.NUM_WORKERS, collate_fn=collate_fn
)
val_loader = DataLoader(
    val_subset, batch_size=config.BATCH_SIZE, shuffle=False,
    num_workers=config.NUM_WORKERS, collate_fn=collate_fn
)
print("✅ COCO DataLoaders created!")

class CocoConfig:
    DATA_ROOT = Path('./data/coco')
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    BATCH_SIZE = 4
    EPOCHS = 2
    LR = 5e-4 # Mask R-CNN은 약간 더 높은 LR을 사용합니다.
    NUM_WORKERS = 0

config = CocoConfig()
print("--- Mask R-CNN Test Configuration ---")
print(f"Device: {config.DEVICE}")



loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
✅ COCO DataLoaders created!
--- Mask R-CNN Test Configuration ---
Device: cuda


In [78]:
# ==============================================================================
# 2. 데이터 로더 생성 (이전과 동일)
# ==============================================================================
full_dataset = CocoDataset(root=config.DATA_ROOT, split='val')
train_size = int(0.1 * len(full_dataset))
val_size = int(0.02 * len(full_dataset))
train_subset, val_subset, _ = torch.utils.data.random_split(
    full_dataset, [train_size, val_size, len(full_dataset) - train_size - val_size]
)
train_loader = DataLoader(
    train_subset, batch_size=config.BATCH_SIZE, shuffle=True,
    num_workers=config.NUM_WORKERS, collate_fn=collate_fn
)
val_loader = DataLoader(
    val_subset, batch_size=config.BATCH_SIZE, shuffle=False,
    num_workers=config.NUM_WORKERS, collate_fn=collate_fn
)
print("✅ COCO DataLoaders created!")

loading annotations into memory...
Done (t=0.63s)
creating index...
index created!
✅ COCO DataLoaders created!


In [79]:
# ==============================================================================
# 3. Mask R-CNN 모델 생성
# ==============================================================================
def get_maskrcnn_model(num_classes):
    # COCO에서 사전 학습된 모델 불러오기
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights='DEFAULT')
    
    # 분류기(classifier)를 새로운 클래스 수에 맞게 교체
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # 마스크 예측기(mask predictor)를 새로운 클래스 수에 맞게 교체
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    
    return model

# COCO는 80개 클래스 + 배경(background) 1개 = 총 81개
model = get_maskrcnn_model(81).to(config.DEVICE)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=config.LR)
print("✅ Mask R-CNN model created!")

✅ Mask R-CNN model created!


In [80]:
def train_epoch_coco(model, criterion, dataloader, optimizer, device, epoch):
    model.train()
    model.head_b.train()
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"COCO Test Epoch {epoch+1}/{config.EPOCHS} [Train]")
    for images, targets in pbar:
        images = images.to(device)
        targets_gpu = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with autocast():
            outputs = model(images)
            loss_dict = criterion(outputs, targets_gpu)
            weighted_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        optimizer.zero_grad()
        scaler.scale(weighted_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += weighted_loss.item()
        pbar.set_postfix({'loss': f'{weighted_loss.item():.4f}'})
    return total_loss / len(dataloader)


def validate_coco(model, dataloader, device):
    model.eval()
    map_metric = MeanAveragePrecision(iou_type="segm")

    with torch.no_grad():
        pbar_val = tqdm(dataloader, desc="[Valid]")
        for images, targets in pbar_val:
            images = list(img.to(device) for img in images)
            
            predictions = model(images)
            
            # --- [핵심 수정] torchmetrics 형식 변환 시 uint8 사용 ---
            preds_for_map = []
            for p in predictions:
                p_cpu = {k: v.cpu() for k, v in p.items()}
                # 예측된 마스크를 thresholding 후 uint8로 변환
                p_cpu['masks'] = (p_cpu['masks'].squeeze(1) > 0.5).to(torch.uint8)
                preds_for_map.append(p_cpu)

            targets_for_map = [{k: v.cpu() for k, v in t.items()} for t in targets]
            
            map_metric.update(preds_for_map, targets_for_map)
            
    map_results = map_metric.compute()
    print(f"\n -> Val mAP: {map_results['map'].item():.4f}")

print("✅ COCO Test functions are ready.")

✅ COCO Test functions are ready.


In [81]:
# ==============================================================================
# 4. Mask R-CNN 학습 루프
# ==============================================================================
print("\n--- 🚀 Starting Mask R-CNN Sanity Check Training 🚀 ---")
for epoch in range(config.EPOCHS):
    model.train()
    pbar = tqdm(train_loader, desc=f"Mask R-CNN Epoch {epoch+1}/{config.EPOCHS} [Train]")
    
    # --- 학습 ---
    for images, targets in pbar:
        images = list(img.to(config.DEVICE) for img in images)
        targets = [{k: v.to(config.DEVICE) for k, v in t.items()} for t in targets]

        # Mask R-CNN은 학습 시 loss 딕셔너리를 직접 반환
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        pbar.set_postfix({'loss': f'{losses.item():.4f}'})

    # --- 검증 ---
    model.eval()
    map_metric = MeanAveragePrecision(iou_type="segm")
    with torch.no_grad():
        pbar_val = tqdm(val_loader, desc="[Valid]")
        for images, targets in pbar_val:
            images = list(img.to(config.DEVICE) for img in images)
            
            predictions = model(images)
            
            # torchmetrics 형식에 맞게 변환
            preds_for_map = [{k: v.cpu() for k, v in p.items()} for p in predictions]
            targets_for_map = [{k: v.cpu() for k, v in t.items()} for t in targets]
            
            map_metric.update(preds_for_map, targets_for_map)
            
    map_results = map_metric.compute()
    print(f"\nEpoch {epoch+1}/{config.EPOCHS} -> mAP: {map_results['map'].item():.4f}")

print("\n--- 🎉 Mask R-CNN Test Complete ---")


--- 🚀 Starting Mask R-CNN Sanity Check Training 🚀 ---


Mask R-CNN Epoch 1/2 [Train]:   0%|          | 0/125 [00:00<?, ?it/s]


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [66]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pathlib import Path
from PIL import Image
import numpy as np
import json
from tqdm import tqdm
from types import SimpleNamespace

# pycocotools와 torchmetrics가 필요합니다.
from pycocotools.coco import COCO
from torchmetrics.detection import MeanAveragePrecision

print("--- 🚀 Starting Final Pre-flight Check... ---")

try:
    # ----------------------------------------------------
    # 1. 설정 (Configuration)
    # ----------------------------------------------------
    class CocoConfig:
        DATA_ROOT = Path('./data/coco')
        DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
        BATCH_SIZE = 2 # 테스트를 위해 작은 배치 크기 사용
        NUM_WORKERS = 0

    config = CocoConfig()

    # ----------------------------------------------------
    # 2. 데이터셋 클래스 및 Collate 함수 (모든 수정사항 반영)
    # ----------------------------------------------------
    class CocoDataset(Dataset):
        def __init__(self, root, split='val'):
            self.root = Path(root)
            self.split = split
            self.images_dir = self.root / f"{split}2017"
            self.coco = COCO(self.root / 'annotations' / f"instances_{split}2017.json")
            self.ids = list(sorted(self.coco.imgs.keys()))
            self.cat_ids = sorted(self.coco.getCatIds())
            self.coco_labels_map = {cat_id: i + 1 for i, cat_id in enumerate(self.cat_ids)}
            self.transform = transforms.ToTensor()

        def __len__(self):
            return len(self.ids)

        def __getitem__(self, idx):
            img_id = self.ids[idx]
            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            coco_anns = self.coco.loadAnns(ann_ids)
            img_info = self.coco.loadImgs(img_id)[0]
            image = Image.open(self.images_dir / img_info['file_name']).convert('RGB')
            
            target = {}
            masks, labels, boxes = [], [], []
            for ann in coco_anns:
                if ann['iscrowd'] == 0 and ann['area'] > 0:
                    masks.append(self.coco.annToMask(ann))
                    labels.append(self.coco_labels_map[ann['category_id']])
                    x, y, w, h = ann['bbox']
                    boxes.append([x, y, x + w, y + h])
            
            image = self.transform(image)
            
            if boxes:
                target['boxes'] = torch.as_tensor(boxes, dtype=torch.float32)
                target['labels'] = torch.as_tensor(labels, dtype=torch.int64)
                target['masks'] = torch.as_tensor(np.stack(masks), dtype=torch.uint8)
            else: 
                target['boxes'] = torch.zeros((0, 4), dtype=torch.float32)
                target['labels'] = torch.zeros((0,), dtype=torch.int64)
                target['masks'] = torch.zeros((0, img_info['height'], img_info['width']), dtype=torch.uint8)
            return image, target

    def collate_fn(batch):
        return tuple(zip(*batch))

    # ----------------------------------------------------
    # 3. 데이터 로더 생성
    # ----------------------------------------------------
    val_dataset = CocoDataset(root=config.DATA_ROOT, split='val')
    val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=config.NUM_WORKERS, collate_fn=collate_fn)
    
    print("✅ Dataset and DataLoader are OK.")
    
    # ----------------------------------------------------
    # 4. 모델 초기화
    # ----------------------------------------------------
    def get_maskrcnn_model(num_classes):
        model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights='DEFAULT')
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
        model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, 256, num_classes)
        return model

    model = get_maskrcnn_model(81).to(config.DEVICE)
    print("✅ Model initialization is OK.")

    # ----------------------------------------------------
    # 5. 최종 검증 로직 테스트
    # ----------------------------------------------------
    images, targets = next(iter(val_loader))
    
    # 학습 모드 테스트
    model.train()
    images_gpu = list(img.to(config.DEVICE) for img in images)
    targets_gpu = [{k: v.to(config.DEVICE) for k, v in t.items()} for t in targets]
    loss_dict = model(images_gpu, targets_gpu)
    losses = sum(loss for loss in loss_dict.values())
    print(f"✅ Training forward pass is OK (Loss: {losses.item():.4f}).")

    # 검증 모드 테스트
    model.eval()
    map_metric = MeanAveragePrecision(iou_type="segm")
    with torch.no_grad():
        predictions = model(images_gpu)
        preds_for_map = []
        for p in predictions:
            p_cpu = {k: v.cpu() for k, v in p.items()}
            p_cpu['masks'] = (p_cpu['masks'].squeeze(1) > 0.5).to(torch.uint8)
            preds_for_map.append(p_cpu)
        targets_for_map = [{k: v.cpu() for k, v in t.items()} for t in targets]
        map_metric.update(preds_for_map, targets_for_map)
        map_results = map_metric.compute()

    print(f"✅ Validation logic and mAP calculation is OK (Sample mAP: {map_results['map']:.4f}).")
    
    print("\n\n✅✅✅ Pre-flight Check PASSED! All components are working correctly. ✅✅✅")

except Exception as e:
    print(f"\n\n❌❌❌ Pre-flight Check FAILED! An error occurred. ❌❌❌")
    import traceback
    traceback.print_exc()

--- 🚀 Starting Final Pre-flight Check... ---
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!
✅ Dataset and DataLoader are OK.
✅ Model initialization is OK.
✅ Training forward pass is OK (Loss: 5.8424).
✅ Validation logic and mAP calculation is OK (Sample mAP: 0.0000).


✅✅✅ Pre-flight Check PASSED! All components are working correctly. ✅✅✅
